In [1]:
import tensorflow as tf
import numpy as np

import model.cnn as cnn
import model.rnn as rnn
import model.classifier as classifier

import data.ops

import time
import matplotlib.pyplot as plt
%matplotlib inline

# Model definition

In [2]:
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(32, [])
input_op, seq_len, label, weight = data.ops.get_batch_ops(batch_size=batch_size, path='./data/small.TFRecord')

cnn_params = {
    'out_dims' : [10],
    'kernel_sizes' : 32,
    'pool_sizes' : 10
}
rnn_params = {
    'rnn_sizes' : [10],
    'time_steps' : 100
}
fc_params = {
    'fc_sizes' : []
}

c = cnn.get_output(seq_len=seq_len, input_op=input_op, **cnn_params)
r = rnn.get_model(batch_size=batch_size, seq_len=seq_len, input_op=c, **rnn_params)
logits, pred = classifier.get_logits_and_pred(input_op=r.last_output, **fc_params)



CNN--cnn10x32
Tensor("CNN/conv_module/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)

RNN--rnn--steps100--sizes10
LSTMStateTuple(c=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_2:0' shape=(?, 10) dtype=float32>, h=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_3:0' shape=(?, 10) dtype=float32>)

FC--fc
Tensor("classifier/fully_connected/logits/BiasAdd:0", shape=(?, 4), dtype=float32)
Tensor("classifier/pred:0", shape=(?, 4), dtype=float32)


# Sparse, weighted softmax loss

In [3]:
loss = tf.losses.sparse_softmax_cross_entropy(label, logits, weight)
unweighted_loss = tf.losses.sparse_softmax_cross_entropy(label, logits)

In [4]:
def measure_time(op, n_times=10):
    with tf.Session() as sess:
        print('Sess started')
        coord = tf.train.Coordinator()
        tf.global_variables_initializer().run()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        init_state = sess.run(r.zero_state)
        feed_dict = {r.init_state : init_state}
        print('Evaluating')
        for _ in range(n_times):
            t = time.time()
            test_output = sess.run(op, feed_dict)
            print(test_output, 'Eval time:', time.time() - t)
            
        print('Closing threads')
        coord.request_stop()
        coord.join(threads)
        sess.close()

        return test_output

In [5]:
measure_time([loss, unweighted_loss])

Sess started
Evaluating
[0.63111538, 1.399884] Eval time: 2.642584800720215
[0.63263184, 1.3917959] Eval time: 0.48341822624206543
[0.63062447, 1.4083436] Eval time: 0.4236116409301758
[0.63225257, 1.4131725] Eval time: 0.43683934211730957
[0.64441735, 1.4087191] Eval time: 0.4515969753265381
[0.63168967, 1.3782289] Eval time: 0.39233875274658203
[0.62079853, 1.3822248] Eval time: 0.472182035446167
[0.63041484, 1.4059429] Eval time: 0.465831995010376
[0.65027702, 1.4190985] Eval time: 0.4581892490386963
[0.63651252, 1.3986623] Eval time: 0.46692919731140137
Closing threads


[0.63651252, 1.3986623]

# Train operator

In [6]:
learning_rate = tf.Variable(initial_value=.05, trainable=False, name='learning_rate')
global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
grad_clip = tf.Variable(initial_value=3., trainable=False, name='grad_clip')

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss)
    with tf.name_scope('gradient_clipping'):
        capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) 
                      for grad, var in gvs]
        
    opt = optimizer.apply_gradients(capped_gvs, global_step)

In [7]:
measure_time([loss, unweighted_loss, opt], n_times=100)

Sess started
Evaluating
[0.63634324, 1.3790075, None] Eval time: 3.5148799419403076
[0.57901454, 1.3113109, None] Eval time: 1.4440007209777832
[0.50173265, 1.2342627, None] Eval time: 1.5418884754180908
[0.38051069, 1.0149763, None] Eval time: 1.4092023372650146
[0.32744494, 1.042408, None] Eval time: 1.6080615520477295
[0.24281341, 0.97298557, None] Eval time: 1.568976879119873
[0.26619786, 1.1944469, None] Eval time: 1.768803358078003
[0.25964981, 1.2531438, None] Eval time: 1.4534945487976074
[0.21640766, 1.1407042, None] Eval time: 1.6723036766052246
[0.24752289, 1.1663011, None] Eval time: 1.5473780632019043
[0.20086487, 1.0661288, None] Eval time: 1.4607608318328857
[0.24243683, 1.1949792, None] Eval time: 1.56355881690979
[0.21813297, 1.1018785, None] Eval time: 1.5268046855926514
[0.2327787, 1.16009, None] Eval time: 1.514350414276123
[0.23085181, 1.0898123, None] Eval time: 1.5247688293457031
[0.20573896, 1.2300351, None] Eval time: 1.5255327224731445
[0.23222397, 1.1322289, 

[0.21181808, 0.91633737, None]